## Get json data from Mongo to feed into Neo4j


In [14]:
import pymongo
from utils import mongo_utils
import json

creds = {
    'user': 'root',
    'password': 'example',
    'db': 'LUapp',
    'host':'localhost',
    'port': '27017'
}

In [2]:
mongoCon = mongo_utils.Mongo_Connector(creds=creds)


In [3]:
mydb = mongoCon.client[creds['db']]
mycol = mydb["companies"]


In [17]:
myquery =  [
    {"$addFields": {"size_sim_comps": {"$size": "$similar_companies"}}},
    {"$match": {"size_sim_comps": {"$gt": 0}}},
    {"$project": {"name": 1, "similar_companies": 1, "_id": 0}}
]

In [18]:
mydoc = mycol.aggregate(myquery)

In [19]:
similar_companies = []
for x in mydoc:
  similar_companies.append(x)


In [20]:
len(similar_companies)

2331

In [21]:
similar_companies[0]

{'name': "'47",
 'similar_companies': [{'name': 'New Era Cap',
   'link': 'https://www.linkedin.com/company/new-era-cap',
   'industry': 'Apparel & Fashion',
   'location': 'Buffalo, New York'},
  {'name': 'Life is Good',
   'link': 'https://www.linkedin.com/company/life-is-good',
   'industry': 'Apparel & Fashion',
   'location': 'Boston, MA'},
  {'name': 'Mitchell & Ness Nostalgia Co.',
   'link': 'https://www.linkedin.com/company/mitchell-&-ness',
   'industry': 'Apparel & Fashion',
   'location': None},
  {'name': 'Fanatics, Inc.',
   'link': 'https://www.linkedin.com/company/fanatics-inc-',
   'industry': 'Internet',
   'location': 'Jacksonville, FL., .'},
  {'name': 'New Balance',
   'link': 'https://www.linkedin.com/company/new-balance',
   'industry': 'Sporting Goods',
   'location': 'Brighton, MA'},
  {'name': 'Nike',
   'link': 'https://www.linkedin.com/company/nike',
   'industry': 'Sporting Goods',
   'location': 'Beaverton, OR'},
  {'name': 'Converse',
   'link': 'https://

In [22]:
with open('../data/similar_companies.json','w') as f:
    json.dump(similar_companies, f)

## Locations json

In [28]:
myquery =  [
    {"$addFields": {"size_loc": {"$size": "$locations"}}},
    {"$match": {"size_loc": {"$gt": 0}}},
    {"$project": {"name": 1, "locations": 1, "_id": 0}}
]

In [29]:
mydb = mongoCon.client[creds['db']]
mycol = mydb["companies"]
mydoc = mycol.aggregate(myquery)

In [30]:
company_locations = []
for x in mydoc:
  company_locations.append(x)
len(company_locations)

10000

In [31]:
with open('../data/company_locations.json','w') as f:
    json.dump(company_locations, f)

## Company Specialities

In [32]:
myquery = [
    {"$addFields": {"size_spec": {"$size": "$specialities"}}},
    {"$match": {"size_spec": {"$gt": 0}}},
    {"$project": {"name": 1, "specialities": 1, "_id": 0}}
]

In [33]:
mydb = mongoCon.client[creds['db']]
mycol = mydb["companies"]
mydoc = mycol.aggregate(myquery)

In [34]:
company_spec = []
for x in mydoc:
  company_spec.append(x)
len(company_spec)

6792

In [35]:
with open('../data/company_spec.json','w') as f:
    json.dump(company_spec, f)

# Load in Neo4j

In [25]:
from neo4j import GraphDatabase

In [42]:
creds = {
    'host': 'localhost',
    'port': '7687',
    'user': 'neo4j',
    'password': 'neo4j1'
}

In [43]:
from utils import neo4j_utils

In [44]:
neoCon = neo4j_utils.neo_connector(creds=creds)

In [45]:
with neoCon.driver.session() as s:
    result = s.run("match (n) return count(n);")
    values = []
    for ix, record in enumerate(result):
        values.append(record.values())
    info = result.consume()  # discard

info, values

(<neo4j._work.summary.ResultSummary at 0x23997891090>, [[0]])

In [46]:
with neoCon.driver.session() as s:
    result = s.run("create (c:Company {name: 'Abdullah'});")
    values = []
    for ix, record in enumerate(result):
        values.append(record.values())
    info = result.consume()  # discard

info, values

(<neo4j._work.summary.ResultSummary at 0x239988c79d0>, [])

In [47]:
neoCon.deleteAllNodes()